# Canadian Rental Prices Regression Model

## Introduction

We're going to work with the data preprocessed in [this notebook](DataPreprocessing.ipynb), which contains rental price data associated with multiple factors like the location, type of rental, number of rooms, etc. 

The price is our target variable. We're going to explore different regression models before choosing a final regression model that we will use to predict new rental prices as accurately as possible.

In [4]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy.stats as stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [5]:
# Load data
with open('Data/preprocessed_data.pkl', 'rb') as handle:
    data = pickle.load(handle)

pc_frequency_map = data['pc_frequency_map']
df = data['df']

df.shape

(18832, 11)

### Create helper functions

In [7]:
def residual_analysis_plots(model):
    '''Plots the residuals and the Q-Q Plot in the same figure'''
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, sharex=False, sharey=False, figsize=(15, 6))
    
    sns.scatterplot(x=model.predict(), y=model.resid, ax=ax1)
    ax1.set_title("Residual Plot")
    ax1.set_xlabel("Prediction")
    ax1.set_ylabel("Residuals")
    ax1.axhline(0, linestyle="--", color="orange")

    ax2.hist(model.resid, bins=30)
    ax2.set_title("Distribution of the residuals")
    
    stats.probplot(model.resid, dist="norm", plot=ax3)
    ax3.set_title("Normal Q-Q Plot")  

    plt.tight_layout()
    plt.show()

In [8]:
def eval_performance(test, predictions):
    MAE = mean_absolute_error(test, predictions)
    MSE = mean_squared_error(test, predictions)
    RMSE = np.sqrt(MSE)
    pred_mean = predictions.mean()
    ratio = RMSE / pred_mean
    
    metrics = {
        'MAE': MAE,
        'MSE': MSE,
        'RMSE': RMSE,
        'Predictions Mean': pred_mean,
        'RMSE to mean ratio': ratio
    }

    df = pd.DataFrame(metrics.items(), columns=['Metric', 'Value'])
    df['Value'] = df['Value'].apply(lambda x: '%.4f' % x)
    
    return df

In [9]:
def compare_performances(initial_perf, current_perf):
    df = pd.merge(initial_perf, current_perf, on='Metric', suffixes=['_initial', '_current'])
    df['Difference'] = df['Value_current'].astype('float') - df['Value_initial'].astype('float')
    return df

In [10]:
def find_optimal_degree():
    train_rmse_errors = []
    test_rmse_errors = []
    degrees = []
    
    for d in range(1, 11):
        polynomial_converter = PolynomialFeatures(degree=d, include_bias=False)
        poly_features = polynomial_converter.fit_transform(X)
        
        X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.15, random_state=42)
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        model = LinearRegression()
        model.fit(X_train_scaled, y_train)
        
        train_pred = model.predict(X_train_scaled)
        test_pred = model.predict(X_test_scaled)
        
        train_RMSE = np.sqrt(mean_squared_error(y_train, train_pred))
        test_RMSE = np.sqrt(mean_squared_error(y_test, test_pred))
        
        train_rmse_errors.append(train_RMSE)
        test_rmse_errors.append(test_RMSE)
        degrees.append(d)

    results_df = pd.DataFrame({
        'Degree': degrees,
        'Train RMSE': train_rmse_errors.apply(lambda x: '%.4f' % x),
        'Test RMSE': test_rmse_errors.apply(lambda x: '%.4f' % x),
    })

    return results_df.sort_values(by=['Test RMSE', 'Train RMSE']).head(1)

---

## 1. First Model: Linear Regression

In [13]:
# Separate features and target variable
X = df.drop('price', axis=1)
y = df['price']

### 1.1 Base model

In [15]:
# Fit model
X_constant = sm.add_constant(X)
base_model = sm.OLS(y, X_constant).fit()
base_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     936.3
Date:                Sat, 01 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:35:06   Log-Likelihood:            -1.5168e+05
No. Observations:               18832   AIC:                         3.034e+05
Df Residuals:                   18821   BIC:                         3.035e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                601.9865     26.519     22.700      0.000     550.007     653.966
beds                 171.6820      8.283     20.726      0.000     155.446     187.918
baths                477.7080     12.728     37.532      0.000     452.760     502.656
sq_feet                0.2911      0.015     19.421      0.000       0.262       0.320
cats                -106.0581     22.628     -4.687      0.000    -150.412     -61.705
dogs                 199.6885     21.862      9.134      0.000     156.837     242.540
postal_code            0.4785      0.064      7.500      0.000       0.353       0.604
type_Apartment       368.2307     16.783     21.941      0.000     335.334     401.127
type_Basement       -184.7675     28.406     -6.504      0.000    -240.446    -129.089
type_House            83.4931     29.510      2.829      0.005      25.650     141.336
type_Room For Rent -1223.7033     40.669    -30.090      0.000   -1303.418   -1143.989
==============================================================================
Omnibus:                    22780.249   Durbin-Watson:                   0.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         15166388.964
Skew:                           5.874   Prob(JB):                         0.00
Kurtosis:                     141.530   Cond. No.                     8.34e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.34e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
# Evaluate performance
y_pred = base_model.predict(X_constant)
base_perf = eval_performance(y, y_pred)
base_perf

,Metric,Value
0,MAE,497.4344
1,MSE,580201.2375
2,RMSE,761.7094
3,Predictions Mean,2156.9496
4,RMSE to mean ratio,0.3531


**Observations**

The R-squared is 0.332, which indicates that 33.2% of the variance of the price is explained by the model, which suggests a poor fit.
Based on the p-values, all features seem to be statistically significant.
The Durbin-Watson score of 0.959 indicates that the errors are not independent.

Compared to the mean values, the MAE and RMSE are pretty high, which indicates our model doesn't perform really well.

### 1.2 Model refinement

In [19]:
# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=scaler.get_feature_names_out())

In [20]:
# Fit the model
X_constant = sm.add_constant(X_scaled)
model_scaled = sm.OLS(y, X_constant).fit()
model_scaled.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     936.3
Date:                Sat, 01 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:35:06   Log-Likelihood:            -1.5168e+05
No. Observations:               18832   AIC:                         3.034e+05
Df Residuals:                   18821   BIC:                         3.035e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               2156.9496      5.552    388.483      0.000    2146.067    2167.832
beds                 165.9403      8.006     20.726      0.000     150.247     181.633
baths                301.8346      8.042     37.532      0.000     286.071     317.598
sq_feet              141.1529      7.268     19.421      0.000     126.907     155.399
cats                 -49.3833     10.536     -4.687      0.000     -70.035     -28.731
dogs                  95.0115     10.402      9.134      0.000      74.623     115.400
postal_code           43.0953      5.746      7.500      0.000      31.832      54.359
type_Apartment       168.7905      7.693     21.941      0.000     153.711     183.870
type_Basement        -41.4900      6.379     -6.504      0.000     -53.993     -28.987
type_House            18.7838      6.639      2.829      0.005       5.771      31.797
type_Room For Rent  -178.1565      5.921    -30.090      0.000    -189.762    -166.551
==============================================================================
Omnibus:                    22780.249   Durbin-Watson:                   0.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         15166388.964
Skew:                           5.874   Prob(JB):                         0.00
Kurtosis:                     141.530   Cond. No.                         4.49
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
# Evaluate performance
y_pred = model_scaled.predict(X_constant)
compare_performances(base_perf, eval_performance(y, y_pred))

,Metric,Value_initial,Value_current,Difference
0,MAE,497.4344,497.4344,0.0
1,MSE,580201.2375,580201.2375,0.0
2,RMSE,761.7094,761.7094,0.0
3,Predictions Mean,2156.9496,2156.9496,0.0
4,RMSE to mean ratio,0.3531,0.3531,0.0


**Observations**

Scaling the data doesn't make any difference in the performance of the model.

In [67]:
# Transform y into log(y)
y_log = np.log(y)
y_log = y_log.rename('log(price)')
X_constant = sm.add_constant(X)
model_log = sm.OLS(y_log, X_constant).fit()
model_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             log(price)   R-squared:                       0.316
Model:                            OLS   Adj. R-squared:                  0.315
Method:                 Least Squares   F-statistic:                     868.1
Date:                Sat, 01 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:56:13   Log-Likelihood:                -8011.3
No. Observations:               18832   AIC:                         1.604e+04
Df Residuals:                   18821   BIC:                         1.613e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  6.9858      0.013    541.913      0.000       6.961       7.011
beds                   0.1346      0.004     33.431      0.000       0.127       0.143
baths                  0.2138      0.006     34.550      0.000       0.202       0.226
sq_feet            -5.828e-05   7.29e-06     -8.000      0.000   -7.26e-05    -4.4e-05
cats                  -0.0696      0.011     -6.328      0.000      -0.091      -0.048
dogs                   0.0909      0.011      8.551      0.000       0.070       0.112
postal_code            0.0003    3.1e-05      9.328      0.000       0.000       0.000
type_Apartment         0.1714      0.008     21.014      0.000       0.155       0.187
type_Basement         -0.0763      0.014     -5.526      0.000      -0.103      -0.049
type_House             0.0318      0.014      2.217      0.027       0.004       0.060
type_Room For Rent    -0.7513      0.020    -38.003      0.000      -0.790      -0.713
==============================================================================
Omnibus:                    18959.374   Durbin-Watson:                   0.764
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3531237.268
Skew:                          -4.540   Prob(JB):                         0.00
Kurtosis:                      69.467   Cond. No.                     8.34e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.34e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [69]:
# Evaluate performance
y_pred = model_log.predict(X_constant)
display(compare_performances(base_perf, eval_performance(y_log, y_pred)))
base_perf = eval_performance(y_log, y_pred)

,Metric,Value_initial,Value_current,Difference
0,MAE,497.4344,0.2389,-497.1955
1,MSE,580201.2375,0.1371,-580201.1004
2,RMSE,761.7094,0.3703,-761.3391
3,Predictions Mean,2156.9496,7.5947,-2149.3549
4,RMSE to mean ratio,0.3531,0.0488,-0.3043


Transforming the target variable decreases our model performance in terms of R-squared (0.332 to 0.316), but improves the RMSE compared to the mean. Let's keep log(y) instead of y.

In [72]:
# Assess multicollinearity
vif_scores = pd.Series(
    [vif(X.values, i) for i in range(X.shape[1])],
    index=X.columns
)
vif_scores

beds                   8.199084
baths                 10.948939
sq_feet                7.354206
cats                  11.120170
dogs                  10.132674
postal_code            2.124948
type_Apartment         3.592624
type_Basement          1.155050
type_House             1.509957
type_Room For Rent     1.116085
dtype: float64

In [74]:
# Remove problematic features
columns_to_keep = list(vif_scores[vif_scores <= 5].index)

In [76]:
# Re-fit model
X_vif = X[columns_to_keep]
X_constant = sm.add_constant(X_vif)
model_vif = sm.OLS(y_log, X_constant).fit()
model_vif.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             log(price)   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     569.1
Date:                Sat, 01 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:56:35   Log-Likelihood:                -10257.
No. Observations:               18832   AIC:                         2.053e+04
Df Residuals:                   18826   BIC:                         2.057e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  7.5834      0.009    876.369      0.000       7.566       7.600
postal_code            0.0002   3.44e-05      7.205      0.000       0.000       0.000
type_Apartment         0.0019      0.008      0.228      0.820      -0.014       0.018
type_Basement         -0.2888      0.015    -19.137      0.000      -0.318      -0.259
type_House             0.3507      0.015     23.256      0.000       0.321       0.380
type_Room For Rent    -0.8403      0.022    -38.335      0.000      -0.883      -0.797
==============================================================================
Omnibus:                    17569.623   Durbin-Watson:                   0.864
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2449550.892
Skew:                          -4.061   Prob(JB):                         0.00
Kurtosis:                      58.279   Cond. No.                     1.03e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.03e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [80]:
y_pred = model_vif.predict(X_constant)
compare_performances(base_perf, eval_performance(y_log, y_pred))

,Metric,Value_initial,Value_current,Difference
0,MAE,0.2389,0.2692,0.0303
1,MSE,0.1371,0.1740,0.0369
2,RMSE,0.3703,0.4172,0.0469
3,Predictions Mean,7.5947,7.5947,0.0000
4,RMSE to mean ratio,0.0488,0.0549,0.0061


**Observations**

Removing those features made our model worse in terms of R-squared (0.332 to 0.131). I's not a good idea to remove them.

### 1.3 Summary

---

## 2. Polynomial Regression

### 2.1 Base model

### 2.2 Model refinement

### 2.3 Summary

## 3. Regularization, cross-validation and grid search

### 3.1 Ridge Cross validation

### 3.2 ElasticNet Grid Search

### 3.3 ElasticNet Cross Validation

### 3.4 Lasso Cross Validation

### 3.5 Summary

## 4. Final Model

## 4.1 Interpretation

## 4.2 Make Predictions

## End